<a href="https://colab.research.google.com/github/brianellis1997/Sarcasm_Detection/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [1]:
!git clone https://github.com/brianellis1997/Sarcasm_Detection.git

Cloning into 'Sarcasm_Detection'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 194 (delta 106), reused 115 (delta 62), pack-reused 3
Receiving objects: 100% (194/194), 4.36 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_bal = pd.read_csv('/content/drive/MyDrive/Sarcasm_Data/Train_Balanced.csv')   # Make sure path is correct in your google drive
train_bal.head()

,Unnamed: 0,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,522142,0,"I personally wasn't a huge Garrosh fan, I've a...",cromemako83,AskReddit,2,2,0,2015-07-01,2015-07-11 01:55:53,Fuck Vol'jin. Garrosh Hellscream did nothing w...
1,907864,1,you forgot the,_SharkWeek_,AskReddit,1,1,0,2013-03-01,2013-03-14 03:03:46,That's a lie fed to you by the LIEberal media....
2,604170,1,"Nah man, she's clearly an ad carry",jdswift13,leagueoflegends,1,1,0,2015-10-01,2015-10-21 23:22:17,she isnt already?
3,110635,1,This sub in a nutshell.,trickz-M-,GlobalOffensive,1,-1,-1,2016-12-01,2016-12-05 03:50:18,Cloud 9 Qualify! (ONLY C9 FANS ALLOWED(
4,997758,0,Yes... I do.,guriboysf,videos,4,4,0,2010-01-01,2010-01-17 21:32:40,"""so, i hear you have a fat cock."""


In [15]:
# import numpy as np
# from nltk.tokenize import word_tokenize
# from gensim.models import KeyedVectors
# import nltk
# nltk.download('punkt')

# # Load GloVe word vectors
# glove_file = '/content/drive/MyDrive/Sarcasm_Data/glove.6B.100d.txt'  # Adjust the path to your downloaded GloVe file
# word_vectors = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)

# # Function to obtain word vectors for a sentence
# def get_sentence_embedding(sentence):
#     words = word_tokenize(sentence)  # Tokenize sentence into words
#     word_embeddings = []
#     for word in words:
#         try:
#             word_embeddings.append(word_vectors[word])  # Get GloVe vector for word
#         except KeyError:
#             # Handle out-of-vocabulary words
#             pass
#     if len(word_embeddings) == 0:
#         # Handle case where all words are out-of-vocabulary
#         return np.zeros(word_vectors.vector_size)
#     else:
#         # Aggregate word vectors (take sum)
#         return np.sum(word_embeddings, axis=0)

# # Apply to comment and parent comment
# comment_vectors = train_bal['comment'].apply(get_sentence_embedding)
# parent_comment_vectors = train_bal['parent_comment'].apply(get_sentence_embedding)

# Convert comment_vectors and parent_comment_vectors to NumPy arrays
# comment_vectors_np = np.array(comment_vectors.tolist())
# parent_comment_vectors_np = np.array(parent_comment_vectors.tolist())

# # Save the NumPy arrays to files
# np.save('/content/drive/MyDrive/Sarcasm_Data/comment_vectors.npy', comment_vectors_np)
# np.save('/content/drive/MyDrive/Sarcasm_Data/parent_comment_vectors.npy', parent_comment_vectors_np)

In [3]:
# Load the saved NumPy arrays
comment_vectors = np.load('/content/drive/MyDrive/Sarcasm_Data/comment_vectors.npy')
parent_comment_vectors = np.load('/content/drive/MyDrive/Sarcasm_Data/parent_comment_vectors.npy')

In [4]:
print(comment_vectors.shape)
print(parent_comment_vectors.shape)

(808618, 100)
(808618, 100)


In [5]:
print(comment_vectors[0])
print('-'*100)
print(parent_comment_vectors[0])

[-7.95173943e-01  3.14127636e+00  2.78570318e+00 -5.05811024e+00
 -5.42068005e-01  3.63985920e+00 -1.27075803e+00  1.66128492e+00
  3.84848982e-01 -5.39201796e-01  1.56411278e+00 -4.96732265e-01
  3.14057279e+00 -1.09438193e+00  1.24003977e-01 -4.92372990e+00
  1.95057189e+00  1.45621693e+00 -5.86305523e+00  3.62457490e+00
  2.70052624e+00  2.39906836e+00  1.30075109e+00 -1.94248295e+00
  4.43695879e+00  5.67565024e-01 -4.04613018e+00 -4.17990780e+00
  2.51732898e+00 -2.02238202e+00 -4.93887007e-01  7.83402014e+00
  1.92612505e+00  3.37638760e+00  1.19841099e+00  2.33827353e+00
 -1.06820077e-01  4.66233015e+00  1.24715900e+00 -3.10010982e+00
 -3.65968680e+00 -2.88997602e+00  4.48291922e+00 -5.16998005e+00
 -4.32980001e-01 -1.64014041e-01  3.90807080e+00 -3.62764406e+00
  1.86569667e+00 -1.19882402e+01  2.76568204e-01 -2.21814656e+00
  2.40661955e+00  1.25513506e+01 -2.04660821e+00 -3.07382088e+01
 -9.85192060e-02  5.29608965e-01  1.64236794e+01  8.33390999e+00
 -9.90572035e-01  1.01262

In [6]:
# Add the embeddings to train_bal DataFrame
train_bal['comment_embedding'] = comment_vectors.tolist()
train_bal['parent_comment_embedding'] = parent_comment_vectors.tolist()

# Split the data into features and target
X = train_bal[['comment_embedding', 'parent_comment_embedding']]  # Features
y = train_bal['label']  # Target

# Convert lists to numpy arrays
X['comment_embedding'] = X['comment_embedding'].apply(np.array)
X['parent_comment_embedding'] = X['parent_comment_embedding'].apply(np.array)

<ipython-input-6-a3b503bd4c31>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['comment_embedding'] = X['comment_embedding'].apply(np.array)
<ipython-input-6-a3b503bd4c31>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['parent_comment_embedding'] = X['parent_comment_embedding'].apply(np.array)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=22)

scaler = StandardScaler()
X_train_scaled = np.vstack(X_train['comment_embedding'].to_numpy())
X_val_scaled = np.vstack(X_val['comment_embedding'].to_numpy())
X_train = scaler.fit_transform(X_train_scaled)
X_val = scaler.transform(np.vstack(X_val['comment_embedding'].to_numpy()))

In [10]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the model
model = Sequential([
    LSTM(units=128, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               66560     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 66689 (260.50 KB)
Trainable params: 66689 (260.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

# Train the model
history = model.fit([comment_vectors, parent_comment_vectors], y_train, validation_split=0.2,
                    batch_size=64, epochs=20, callbacks=[early_stopping])

In [ ]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(X_val, y_val)
print('Validation Loss:', loss)
print('Validation Accuracy:', accuracy)